In [ ]:
colab = False
if colab:
    from google.colab import drive
    drive.mount('gdrive')
    gdrive_dir = 'cache'

In [ ]:
import os
import sys
import h5py
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from modis_utils.misc import cache_data, restore_data

In [ ]:
data = restore_data(os.path.join('cache', 'boundary_vectors_ALL.h5'))

In [ ]:
train_boundary_vectors = data[0]
val_boundary_vectors = data[1]
test_boundary_vectors = data[2]

In [ ]:
n_points = train_boundary_vectors.shape[1]
n_points

In [ ]:
train_boundary_vectors.shape, val_boundary_vectors.shape, test_boundary_vectors.shape

In [ ]:
def transform(data, scaler):
    old_shape = data.shape
    data = data.reshape(old_shape[0], -1)
    if scaler is not None:
        data = scaler.transform(data.astype(np.float))
    #return data.reshape(old_shape)
    return data
  
def transform_standardize(data, mean, std):
    old_shape = data.shape
    data = data.reshape(old_shape[0], -1)
    data = (data - mean)/std
    return data.reshape(old_shape)
    #return data
    
def find_mean_std(data):
    old_shape = data.shape
    data = data.reshape(old_shape[0], -1)
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    std[std == 0] = 1
    #mean = mean.reshape(-1, old_shape[-1])
    #std = std.reshape(-1, old_shape[-1])
    return mean, std

In [ ]:
scaler = None
scale_data = False
if scale_data:
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(train_boundary_vectors.reshape(train_boundary_vectors.shape[0], -1))
    
mean, std = find_mean_std(train_boundary_vectors)
train_boundary_vectors_1 = transform_standardize(train_boundary_vectors, mean, std)
val_boundary_vectors_1 = transform_standardize(val_boundary_vectors, mean, std)
test_boundary_vectors_1 = transform_standardize(test_boundary_vectors, mean, std)

In [ ]:
# normalize the dataset
'''
train_boundary_vectors_scale_1 = transform(train_boundary_vectors, scaler, flatten=False)
val_boundary_vectors_scale_1 = transform(val_boundary_vectors, scaler, flatten=False)
test_boundary_vectors_scale_1 = transform(test_boundary_vectors, scaler, flatten=False)
'''

In [ ]:
train_boundary_vectors_1.shape

In [ ]:
def create_dataset(boundary_vectors_scale, timesteps):
    data_X = []
    data_Y = []
    for i in range(len(boundary_vectors_scale) - timesteps):
        data_x = boundary_vectors_scale[i:(i+timesteps)]
        data_y = boundary_vectors_scale[i + timesteps]
        data_X.append(data_x)
        data_Y.append(data_y)
    return np.asarray(data_X), np.asarray(data_Y)

In [ ]:
timesteps = 50
train_X, train_Y = create_dataset(train_boundary_vectors_1, timesteps)
val_X, val_Y = create_dataset(np.concatenate(
    [train_boundary_vectors_1[-timesteps:], val_boundary_vectors_1]),
                              timesteps)
test_X, test_Y = create_dataset(np.concatenate(
    [val_boundary_vectors_1[-timesteps:], test_boundary_vectors_1]),
                                timesteps)

In [ ]:
train_X.shape, train_Y.shape, val_X.shape, val_Y.shape, test_X.shape, test_Y.shape

In [ ]:
def create_graph_matrix(n_points_on_boundary):
    def calc_arc_distance(a, b, n):
        diff = np.abs(a-b)
        if diff > n//2:
            diff = n - diff
        return diff
    
    n = n_points_on_boundary
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = calc_arc_distance(i, j, n)
    return mat.astype(np.float32)

def create_graph_matrix_1(n_points_on_boundary):
    def calc_arc_distance(a, b, n):
        diff = np.abs(a-b)
        if diff > n//2:
            diff = n - diff
        return diff
    
    n = n_points_on_boundary
    mat = np.zeros((2*n, 2*n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = calc_arc_distance(i, j, n)
    mat[n:2*n, n:2*n] = mat[:n, :n]
    for i in range(n):
        for j in range(n, 2*n):
            mat[i,j] = mat[i, j - n]
    mat[n:2*n, :n] = mat[:n, n:2*n]
    return mat.astype(np.float32)

In [ ]:
mat = create_graph_matrix(n_points)
mat.shape

In [ ]:
A = np.divide(mat, n_points)

In [ ]:
A.shape

In [ ]:
#from grnn.model_keras import GRNN

In [ ]:
import sys
import keras
import tensorflow as tf
from keras import backend as K
from keras.optimizers import adam
from keras.models import Model, Input
from keras.layers import Layer, RNN, GRUCell, GRU
from keras.layers import deserialize as deserialize_layer
from keras import activations, initializers, regularizers, constraints

def tf_print(tensor, message=None):
    def print_message(x):
        sys.stdout.write(message + " %s\n" % x)
        return x

    prints = [tf.py_func(print_message, [tensor], tensor.dtype)]
    with tf.control_dependencies(prints):
        op = tensor + 1
        op = tf.identity(op)
    return op

# GRNNCell
class GRUCellKeepDim(GRUCell):
    def __init__(self, n_dims, units,
                 activation='tanh',
                 recurrent_activation='hard_sigmoid',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=1,
                 reset_after=False,
                 **kwargs):
        super(GRUCellKeepDim, self).__init__(units, activation, recurrent_activation)
        self.n_dims = n_dims
    
    def build(self, input_shape):
        self.last_kernel = self.add_weight(shape=(self.units, self.n_dims),
                                           name='last_kernel',
                                           initializer=self.kernel_initializer,
                                           regularizer=self.kernel_regularizer,
                                           constraint=self.kernel_constraint)
        super(GRUCellKeepDim, self).build(input_shape)
        
    def call(self, inputs, states, training=None):
        output, states = super(GRUCellKeepDim, self).call(inputs, states, training)
        output = K.dot(output, self.last_kernel)
        return output, states
    
class GRUKeepDim(GRU):
    def __init__(self, n_dims, units,
                 activation='tanh',
                 recurrent_activation='hard_sigmoid',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=1,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 reset_after=False,
                 **kwargs):
        if implementation == 0:
            warnings.warn('`implementation=0` has been deprecated, '
                          'and now defaults to `implementation=1`.'
                          'Please update your layer call.')
        if K.backend() == 'theano' and (dropout or recurrent_dropout):
            warnings.warn(
                'RNN dropout is no longer supported with the Theano backend '
                'due to technical limitations. '
                'You can either set `dropout` and `recurrent_dropout` to 0, '
                'or use the TensorFlow backend.')
            dropout = 0.
            recurrent_dropout = 0.

        cell = GRUCellKeepDim(n_dims, units,
                              activation=activation,
                              recurrent_activation=recurrent_activation,
                              use_bias=use_bias,
                              kernel_initializer=kernel_initializer,
                              recurrent_initializer=recurrent_initializer,
                              bias_initializer=bias_initializer,
                              kernel_regularizer=kernel_regularizer,
                              recurrent_regularizer=recurrent_regularizer,
                              bias_regularizer=bias_regularizer,
                              kernel_constraint=kernel_constraint,
                              recurrent_constraint=recurrent_constraint,
                              bias_constraint=bias_constraint,
                              dropout=dropout,
                              recurrent_dropout=recurrent_dropout,
                              implementation=implementation,
                              reset_after=reset_after)
        RNN.__init__(self, cell,
                     return_sequences=return_sequences,
                     return_state=return_state,
                     go_backwards=go_backwards,
                     stateful=stateful,
                     unroll=unroll,
                     **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        
    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        if hasattr(self.cell.state_size, '__len__'):
            state_size = self.cell.state_size
        else:
            state_size = [self.cell.state_size]
        output_dim = self.cell.n_dims

        if self.return_sequences:
            output_shape = (input_shape[0], input_shape[1], output_dim)
        else:
            output_shape = (input_shape[0], output_dim)

        if self.return_state:
            state_shape = [(input_shape[0], dim) for dim in state_size]
            return [output_shape] + state_shape
        else:
            return output_shape

    def call(self, inputs, mask=None, training=None, initial_state=None):
        self.cell._dropout_mask = None
        self.cell._recurrent_dropout_mask = None
        return super(GRUKeepDim, self).call(inputs,
                                             mask=mask,
                                             training=training,
                                             initial_state=initial_state)


class GRNNCell(Layer):
    def __init__(cells, **kwargs):
        super(GRNNCell, self).__init__(**kwargs)
        self.cells = cells
        
    def __init__(self, cells, **kwargs):
        for cell in cells:
            if not hasattr(cell, 'call'):
                raise ValueError('All cells must have a `call` method. '
                                 'received cells:', cells)
            if not hasattr(cell, 'state_size'):
                raise ValueError('All cells must have a '
                                 '`state_size` attribute. '
                                 'received cells:', cells)
        self.cells = cells
        super(GRNNCell, self).__init__(**kwargs)

    @property
    def state_size(self):
        state_size = []
        for cell in self.cells:
            if hasattr(cell.state_size, '__len__'):
                state_size += list(cell.state_size)
            else:
                state_size.append(cell.state_size)
        return tuple(state_size)

    def call(self, inputs, states, constants=None, **kwargs):
        # Recover per-cell states.
        nested_states = []
        for cell in self.cells:
            if hasattr(cell.state_size, '__len__'):
                nested_states.append(states[:len(cell.state_size)])
                states = states[len(cell.state_size):]
            else:
                nested_states.append([states[0]])
                states = states[1:]

        # Call the cells in order and store the returned states.
        new_nested_states = []
        for cell, states in zip(self.cells, nested_states):
            if has_arg(cell.call, 'constants'):
                inputs, states = cell.call(inputs, states,
                                           constants=constants,
                                           **kwargs)
            else:
                inputs, states = cell.call(inputs, states, **kwargs)
            new_nested_states.append(states)

        # Format the new states as a flat list
        # in reverse cell order.
        states = []
        for cell_states in new_nested_states:
            states += cell_states
        return inputs, states

    def build(self, input_shape):
        timesteps = input_shape[1]
        n_dims=input_shape[-1]
        cell_input_shape = (None, timesteps, n_dims)
        for cell in self.cells:
            if isinstance(cell, Layer):
                cell.build(cell_input_shape)
        self.built = True

    def get_config(self):
        cells = []
        for cell in self.cells:
            cells.append({'class_name': cell.__class__.__name__,
                          'config': cell.get_config()})
        config = {'cells': cells}
        base_config = super(GRNNCell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config, custom_objects=None):
        cells = []
        for cell_config in config.pop('cells'):
            cells.append(deserialize_layer(cell_config,
                                           custom_objects=custom_objects))
        return cls(cells, **config)

    @property
    def trainable_weights(self):
        if not self.trainable:
            return []
        weights = []
        for cell in self.cells:
            if isinstance(cell, Layer):
                weights += cell.trainable_weights
        return weights

    @property
    def non_trainable_weights(self):
        weights = []
        for cell in self.cells:
            if isinstance(cell, Layer):
                weights += cell.non_trainable_weights
        if not self.trainable:
            trainable_weights = []
            for cell in self.cells:
                if isinstance(cell, Layer):
                    trainable_weights += cell.trainable_weights
            return trainable_weights + weights
        return weights

    def get_weights(self):
        """Retrieves the weights of the model.

        # Returns
            A flat list of Numpy arrays.
        """
        weights = []
        for cell in self.cells:
            if isinstance(cell, Layer):
                weights += cell.weights
        return K.batch_get_value(weights)

    def set_weights(self, weights):
        """Sets the weights of the model.

        # Arguments
            weights: A list of Numpy arrays with shapes and types matching
                the output of `model.get_weights()`.
        """
        tuples = []
        for cell in self.cells:
            if isinstance(cell, Layer):
                num_param = len(cell.weights)
                weights = weights[:num_param]
                for sw, w in zip(cell.weights, weights):
                    tuples.append((sw, w))
                weights = weights[num_param:]
        K.batch_set_value(tuples)

    @property
    def losses(self):
        losses = []
        for cell in self.cells:
            if isinstance(cell, Layer):
                cell_losses = cell.losses
                losses += cell_losses
        return losses

    def get_losses_for(self, inputs=None):
        losses = []
        for cell in self.cells:
            if isinstance(cell, Layer):
                cell_losses = cell.get_losses_for(inputs)
                losses += cell_losses
        return losses


# GRNN
class GRNN(Layer):
    def __init__(self, n_nodes, n_dims, n_hiddens,
                 activation='tanh',
                 recurrent_activation='hard_sigmoid',
                 return_sequences=False,
                 **kwargs):
        super(GRNN, self).__init__(**kwargs)
        self.n_nodes = n_nodes
        self.n_dims = n_dims
        self.n_hiddens = n_hiddens
        
        cells = []
        for i in range(n_nodes):
            cell = GRUKeepDim(n_dims=n_dims, units=n_hiddens,
                              activation=activation,
                              recurrent_activation=recurrent_activation,
                              return_state=True,
                              return_sequences=return_sequences)
            cells.append(cell)
        
        self.return_sequences = return_sequences
        self.grnn_cell = GRNNCell(cells)
        self.h_state = None
        self.trainable = True
        self._num_constants = None

    def build(self, input_shape):
        # input_shape[0] = (T, N, D)
        # h_state = (T, D, N)
        input_shape = input_shape[0]
        self.timesteps = input_shape[1]
        cell_input_shape = (None, self.timesteps, self.n_dims)
        self.grnn_cell.build(input_shape)
        self.built = True

    def compute_output_shape(self, input_shape):
        # output_shape[0] = (N, D)
        input_shape = input_shape[0]
        cell_input_shape = (input_shape[0], input_shape[-1])
        cell_output_shape = self.grnn_cell.cells[0].compute_output_shape(cell_input_shape)
        output_shape = cell_output_shape[0]
        if self.return_sequences:
            return (None, output_shape[0], self.n_nodes, output_shape[-1]) # (T, N, H)
        else:
            return (None, self.n_nodes, output_shape[-1]) # (N, H)
    
    def get_initial_state(self, inputs):
        # build an all-zero tensor of shape (B, T, H, N)
        # inputs : (B, T, N, D)
        initial_state = K.zeros_like(inputs)  # (B, T, N, D)
        initial_state = K.sum(initial_state, axis=(1, -1))  # (B, N,)
        initial_state = K.expand_dims(initial_state, axis=1)  # (B, 1, N)
        cell = self.grnn_cell.cells[0].cell
        if hasattr(cell.state_size, '__len__'):
            return [K.tile(initial_state, [1, dim, 1])
                    for dim in cell.state_size]
        else:
            return K.tile(initial_state, [1, cell.state_size, 1])

    def call(self, x, training=None):
        print('call')
        # A = (B, N, N)
        x_main = x[0]
        A = x[1]

        if self.h_state is None:
            self.h_state = self.get_initial_state(x_main) # (B, H, N), (B, H, N) x (B, N, N) = (B, H, N)
        S = K.batch_dot(self.h_state, A, axes=(2,1)) # S: (B, H, N)
    
        #TODO: add tf.while_loop
        O_list = []
        H_list = []
        for n in range(self.n_nodes):
            
            n_tf = K.variable(n)
            p = tf_print(n_tf, "n = {}".format(n))
            result = p.eval(session=K.get_session())
            
            cell = self.grnn_cell.cells[n]
            x_n = x_main[:, :, n, :]
            S_n = S[:, :, n]
            cell_output = cell.call(x_n,
                    initial_state=[S_n],
                    training=training)
            
            O = cell_output[0] # O = (B, H), return_sequences: (B, T, H)
            H = cell_output[1] # H = (B, T, H)
            
            O_list.append(K.expand_dims(O, axis=-2))
            H_list.append(K.expand_dims(H, axis=-1))
        
        O = K.concatenate(O_list, axis=-2)
        if training:
            self.h_state = K.concatenate(H_list, axis=-1)
        return O
    
    @property
    def states(self):
        if self.h_state is None:
            if isinstance(self.grnn_cell.state_size, int):
                num_states = 1
            else:
                num_states = len(self.grnn_cell.state_size)
            #return [None for _ in range(num_states)]
            return None
        return self.h_state

    @states.setter
    def states(self, h_state):
        self.h_state = h_state
    
    def get_config(self):
        config = {'return_sequences': self.return_sequences}
        if self._num_constants is not None:
            config['num_constants'] = self._num_constants

        cell_config = self.grnn_cell.get_config()
        config['grnn_cell'] = {'class_name': self.grnn_cell.__class__.__name__,
                               'config': cell_config}
        base_config = super(GRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config, custom_objects=None):
        cell = deserialize_layer(config.pop('grnn_cell'),
                                 custom_objects=custom_objects)
        num_constants = config.pop('num_constants', None)
        layer = cls(cell, **config)
        layer._num_constants = num_constants
        return layer

    @property
    def trainable_weights(self):
        if not self.trainable:
            return []
        if isinstance(self.grnn_cell, Layer):
            return self.grnn_cell.trainable_weights
        return []

    @property
    def non_trainable_weights(self):
        if isinstance(self.grnn_cell, Layer):
            if not self.trainable:
                return self.grnn_cell.weights
            return self.grnn_cell.non_trainable_weights
        return []

    @property
    def losses(self):
        layer_losses = super(GRNN, self).losses
        if isinstance(self.grnn_cell, Layer):
            return self.grnn_cell.losses + layer_losses
        return layer_losses

    def get_losses_for(self, inputs=None):
        if isinstance(self.grnn_cell, Layer):
            cell_losses = self.grnn_cell.get_losses_for(inputs)
            return cell_losses + super(GRNN, self).get_losses_for(inputs)
        return super(GRNN, self).get_losses_for(inputs)
    
def bmm(x, A):
    current_shape = K.shape(x)
    x = K.reshape(x, (-1, current_shape[-1]))
    M = K.dot(x, A)
    return M.reshape(current_shape)

In [5]:
import numpy as np
import keras
import tensorflow as tf
from keras import backend as K
from keras.optimizers import adam
from keras.models import Model, Input
from keras.models import model_from_json
from keras.callbacks import TensorBoard

from grnn_keras import GRNN, GRNNCell, GRUKeepDim, GRUCellKeepDim

In [6]:
def create_model(timesteps, n_nodes, n_dims, n_hiddens, dropout=0., recurrent_dropout=0., cell_type='gru'):
    K.clear_session()
    input_main = Input(shape=(timesteps, n_nodes, n_dims))
    input_aux = Input(shape=(n_nodes, n_nodes), name='A')
    inputs = [input_main, input_aux]
    
    x = GRNN(n_nodes, n_dims, n_hiddens, keep_dims=False, return_sequences=True,
             dropout=dropout, recurrent_dropout=recurrent_dropout, cell_type=cell_type)(inputs)
    x = GRNN(n_nodes, n_hiddens, n_hiddens, keep_dims=False, return_sequences=True,
             dropout=dropout, recurrent_dropout=recurrent_dropout, cell_type=cell_type)([x, input_aux])
    x = GRNN(n_nodes, n_hiddens, n_dims, keep_dims=False, cell_type=cell_type,
             dropout=dropout, recurrent_dropout=recurrent_dropout)([x, input_aux])
    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='mse', optimizer=adam(lr=0.001))
    return model

In [12]:
#model = create_model(1, timesteps, n_points, n_hidden)
batch_size = 10
timesteps = 3
n_nodes = 6
n_dims = 4
n_hiddens = 5
dropout = 0.5
recurrent_dropout = 0.5
cell_type = 'gru'
model = create_model(timesteps, n_nodes, n_dims, n_hiddens, dropout, recurrent_dropout, cell_type)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 6, 4)      0                                            
__________________________________________________________________________________________________
A (InputLayer)                  (None, 6, 6)         0                                            
__________________________________________________________________________________________________
grnn_1 (GRNN)                   (None, None, 6, 5)   900         input_1[0][0]                    
                                                                 A[0][0]                          
__________________________________________________________________________________________________
grnn_2 (GRNN)                   (None, None, 6, 5)   990         grnn_1[0][0]                     
          

In [13]:
#model = create_model(1, timesteps, n_points, n_hidden)
batch_size = 10
timesteps = 3
n_nodes = 6
n_dims = 4
n_hiddens = 5
dropout = 0.5
recurrent_dropout = 0.5
cell_type = 'lstm'
model = create_model(timesteps, n_nodes, n_dims, n_hiddens, dropout, recurrent_dropout, 'lstm')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 6, 4)      0                                            
__________________________________________________________________________________________________
A (InputLayer)                  (None, 6, 6)         0                                            
__________________________________________________________________________________________________
grnn_1 (GRNN)                   (None, None, 6, 5)   1200        input_1[0][0]                    
                                                                 A[0][0]                          
__________________________________________________________________________________________________
grnn_2 (GRNN)                   (None, None, 6, 5)   1320        grnn_1[0][0]                     
          

In [14]:
data_x = np.random.randn(batch_size, timesteps, n_nodes, n_dims)
data_y = np.random.randn(batch_size, n_nodes, n_dims)
a = np.random.randn(batch_size, n_nodes, n_nodes)

In [15]:
model.fit([data_x, a], data_y, batch_size=1, epochs=10)

Epoch 1/10
10/10 [==============================] - 17s 2s/step - loss: 1.0137
Epoch 2/10
10/10 [==============================] - 1s 76ms/step - loss: 1.0115
Epoch 3/10
10/10 [==============================] - 1s 76ms/step - loss: 1.0097
Epoch 4/10
10/10 [==============================] - 1s 82ms/step - loss: 1.0080
Epoch 5/10
10/10 [==============================] - 1s 82ms/step - loss: 1.0062
Epoch 6/10
10/10 [==============================] - 1s 84ms/step - loss: 1.0044
Epoch 7/10
10/10 [==============================] - 1s 85ms/step - loss: 1.0025
Epoch 8/10
10/10 [==============================] - 1s 95ms/step - loss: 1.0006
Epoch 9/10
10/10 [==============================] - 1s 91ms/step - loss: 0.9986
Epoch 10/10
10/10 [==============================] - 1s 82ms/step - loss: 0.9966


In [ ]:
from modis_utils.misc import cache_data, restore_data 

In [34]:
# serialize model to JSON
model_json = model.to_json()
with open("grnn.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
weights = model.get_weights()
cache_data(weights, "grnn_weights.h5")
print("Saved model to disk")

Saved model to disk


In [35]:
'''
model1 = model_from_json(open('grnn.json', 'r').read(), custom_objects={'GRNN': GRNN,
                                                                        'GRNNCell': GRNNCell,
                                                                        'GRUKeepDim': GRUKeepDim,
                                                                        'GRUCellKeepDim': GRUCellKeepDim})
'''                                                                    

"\nmodel1 = model_from_json(open('grnn.json', 'r').read(), custom_objects={'GRNN': GRNN,\n                                                                        'GRNNCell': GRNNCell,\n                                                                        'GRUKeepDim': GRUKeepDim,\n                                                                        'GRUCellKeepDim': GRUCellKeepDim})\n"

In [36]:
model1 = create_model(timesteps, n_nodes, n_dims, n_hiddens)

n = 0 0.0
n = 1 1.0
n = 2 2.0
n = 3 3.0
n = 4 4.0
n = 5 5.0


In [43]:
weights1 = restore_data('grnn_weights.h5')
model1.set_weights(weights1)

ValueError: Cannot feed value of shape (5, 15) for Tensor 'Placeholder_12:0', which has shape '(5, 4)'

In [40]:
test_x = np.random.randn(1, timesteps, n_nodes, n_dims)
test_y = np.random.randn(1, n_nodes, n_dims)
test_a = np.random.randn(1, n_nodes, n_nodes)

In [42]:
model1.predict([test_x, test_a])

array([[[-0.8069762 , -0.27716914, -0.08066209, -0.2173638 ],
        [-0.0471504 ,  0.3077119 , -0.50850165, -0.14706776],
        [ 0.28091013,  0.1313437 , -0.18831247,  0.0615936 ],
        [-0.09314417, -0.18319672, -0.4167047 ,  0.04447975],
        [ 0.29835194,  0.44064412, -0.32845873,  0.87659216],
        [-0.32595405, -0.24016304, -0.07882565,  0.17863667]]],
      dtype=float32)

In [ ]:
model1.evaluate([test_x, test_a], test_y)